# OSRM Nearest Service – Coordinate Snapping Analysis

**Objective:**  
This notebook demonstrates how to use the OSRM *Nearest* service
to snap a geographic coordinate to the closest point
on the routable road network.

## Table of Contents

- [Using the Nearest Service](#Using-the-Nearest-Service)
- [Request Construction](#Request-Construction)
- [Using the Service](#Using-the-Service)
  - [General Parameters](#General-Parameters)
  - [Constructing the Request](#Constructing-the-Request)
- [Batch Snapping Example](#Batch-Snapping-Example)
- [Map Matching with the OSRM Match Service](#Map-Matching-with-the-OSRM-Match-Service)
- [Conclusion](#Conclusion)

## Using the Nearest Service

The Nearest service is designed to snap an arbitrary
geographic coordinate to the closest navigable
point on the road network.

This is particularly useful when working with
GPS data, where raw coordinates may not align
exactly with mapped roads.

## Request Construction

The Nearest endpoint follows this format:

/nearest/{version}/{profile}/{coordinates}

Unlike the Route service, this endpoint returns
the closest routable node instead of computing
a full path between waypoints.

In [ ]:
import folium

from folium import Map
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float


def get_folium_map(center_point: Point, points: list[Point], zoom_level: int = 14) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)

    for point in points:
        folium.Marker(location=[point.latitude, point.longitude],
                      popup='Point').add_to(folium_map)

    return folium_map


point_1 = Point(latitude=53.077000, longitude=8.804000)
center_point = Point(latitude=53.077000, longitude=8.804000)

folium_map = get_folium_map(center_point, [point_1], zoom_level=18)
folium_map

## Using the Service

The Nearest endpoint is used to snap a single
geographic coordinate to the closest navigable
road segment within the routing graph.

This is commonly applied when processing
raw GPS data that may not perfectly align
with mapped street geometries.

### General Parameters

| Parameter     | Description |
|--------------|-------------|
| **service**  | Specifies the OSRM operation being invoked. For this notebook, the value is `nearest`. |
| **version**  | Defines the API version of the OSRM service. For OSRM 5.x installations, this is typically `v1`. |
| **profile**  | Determines the routing profile used for snapping (e.g., `driving`, `bike`, `foot`). This affects how the road network is interpreted. |
| **coordinates** | A single coordinate pair formatted as `{longitude},{latitude}` representing the location to be snapped. |
| **format**   | Specifies the response format. Currently, `json` is supported and used by default. |

### Constructing the Request

The Nearest endpoint follows this pattern:

/nearest/{version}/{profile}/{coordinates}?number={N}

Unlike the Route service, this endpoint expects
only one coordinate and returns one or more
candidate road positions ranked by proximity.

In [ ]:
service = 'nearest'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

We define the coordinate to be snapped and
optionally specify the number of nearby
road candidates to retrieve using the
`number` parameter.

In [ ]:
import requests

coordinates = f'{point_1.longitude},{point_1.latitude}'
number_of_snap_points = 5

url = f"{host}/{service}/{version}/{profile}/{coordinates}?number={number_of_snap_points}"

response = requests.get(url)
response.status_code

A status code of `200` confirms that the
request was successfully processed by the server.

We now parse the JSON response to examine
the snapped location data.

The response contains structured metadata
describing the snapped road location(s).

We inspect the available keys to understand
how the nearest candidates are organized.

In [ ]:
data = response.json()
data.keys()

The response includes a list of snapped waypoint candidates.

The number of returned waypoints depends on the
`number` parameter specified in the request.

To process the snapped locations programmatically,
we iterate through the returned waypoints and extract
their geographic coordinates.

Since OSRM provides coordinates in the format
[longitude, latitude], we convert them into
(latitude, longitude) order for compatibility
with the Folium visualization.

In [ ]:
nearest_points = []
for waypoint in waypoints:
    latitude = waypoint['location'][1]
    longitude = waypoint['location'][0]
    nearest_points.append(Point(latitude=latitude, longitude=longitude))

nearest_points

When visualizing the snapped points, we observe that
the original coordinate is adjusted to align with
the nearest navigable road segment.

Even if the input location lies within a building
or off-road area, the Nearest service projects it
onto the routable street network.

This snapping step is essential because routing
algorithms require coordinates to exist on the
graph representation of the road network.

To illustrate the snapping behavior, we overlay
the returned waypoint candidates on the map.

Each blue marker represents a possible
road-aligned location ranked by proximity
to the original input coordinate.

In [ ]:
folium_map = get_folium_map(center_point, [point_1], zoom_level=18)

for point in nearest_points:
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        color='blue',
                        fill=True,
                        fill_color='blue').add_to(folium_map)

folium_map

## Batch Snapping Example

In this example, we process multiple coordinates
stored in a CSV file.

Some of these points may not lie exactly on
the street network. We use the Nearest service
to snap each coordinate to the closest
routable road segment.

We begin by loading a dataset containing
latitude and longitude values.

This dataset simulates real-world GPS inputs
that may require alignment with the road network.

To test the snapping behavior with different locations,
navigate to the file `nearest_service_data.csv`
and modify the latitude and longitude values accordingly.

In [ ]:
import pandas as pd

data = pd.read_csv('../data/nearest_service_data.csv')
data.head()

To maintain consistency with the previous sections,
we convert each row of the dataset into a structured
`Point` object.

This allows us to handle geographic coordinates
in a standardized format throughout the notebook.

In [ ]:
points = []
for _, row in data.iterrows():
    point = Point(latitude=row['latitude'], longitude=row['longitude'])
    points.append(point)

points[:5]

The map below displays the original coordinates
from the dataset prior to snapping.

This provides a baseline view of the input locations
before they are projected onto the street network.

In [ ]:
folium_map = get_folium_map(points[5], points, zoom_level=18)
folium_map

To streamline the snapping workflow, we define
a reusable function that takes a single `Point`
and returns its corresponding snapped waypoint(s).

This modular approach allows us to apply the
Nearest service consistently across multiple inputs.

Inside the function:

- The coordinate is formatted as `{longitude},{latitude}`
- A request is sent to the Nearest endpoint
- The JSON response is parsed
- Returned waypoint coordinates are converted
  into structured `Point` objects

This ensures that all snapped locations follow
the same internal representation.

We then iterate over all input coordinates
and apply the snapping function to each one.

The resulting snapped waypoints are collected
into a list for further visualization.

In [ ]:
def get_waypoints(point: Point) -> list[Point]:
    coordinates = f'{point.longitude},{point.latitude}'
    number_of_snap_points = 1

    url = f"{host}/{service}/{version}/{profile}/{coordinates}?number={number_of_snap_points}"

    response = requests.get(url)

    data = response.json()
    waypoints = [Point(latitude=point['location'][1], longitude=point['location'][0])
                 for point in data['waypoints']]

    return waypoints


waypoints = []
for point in points:
    waypoints.append(get_waypoints(point))

waypoints[:5]

The black markers shown below represent the
snapped road-aligned coordinates returned
by the Nearest service.

Comparing them with the original input locations
demonstrates how off-network points are projected
onto the closest navigable street segments.

To visually confirm the snapping behavior,
we overlay the returned waypoints on the map.

Each black marker corresponds to a road-network
vertex identified as the nearest valid routing node.

In [ ]:
folium_map = get_folium_map(points[5], points, zoom_level=18)

for waypoint in waypoints:
    for point in waypoint:
        folium.CircleMarker(
            location=[point.latitude, point.longitude],
            radius=5,
            color='black',
            fill=True,
            fill_color='black',
        ).add_to(folium_map)

folium_map

## Map Matching with the OSRM Match Service

To reconstruct a continuous route from a sequence
of snapped GPS points, we use the OSRM *Match* service.

Unlike the Route service, which computes an optimal
path between waypoints, the Match service aligns
a series of recorded coordinates to the most
probable path on the road network.

We first build a coordinate string in
`{longitude},{latitude}` format using
the snapped points obtained earlier.

The Match endpoint follows this structure:

/match/{version}/{profile}/{coordinates}

This service performs map matching by projecting
a trace of GPS points onto the routing graph
and reconstructing a coherent path.

After sending the request, the response contains
a list of possible matchings.

Each matching includes a reconstructed geometry
representing the most likely path that connects
the provided trace points.

The returned geometry is formatted as
[longitude, latitude] pairs.

Since Folium expects coordinates in
[latitude, longitude] order, we convert
each point before visualization.

Finally, we render the matched route as a
smooth polyline on the map.

This blue line represents the reconstructed
path inferred by the OSRM matching algorithm
based on the input GPS trace.  

In [ ]:
# ---------------- OSRM MATCH (FIXED) ----------------

# 1️⃣ Build coordinates string (lon,lat)
trace_coordinates = ";".join(
    [f"{p.longitude},{p.latitude}" for p in snapped_points]
)

# 2️⃣ Call MATCH service (THIS IS THE KEY)
match_url = (
    f"{host}/match/v1/{profile}/{trace_coordinates}"
    "?overview=full&geometries=geojson"
)

match_response = requests.get(match_url)
match_data = match_response.json()

# 3️⃣ Extract matched geometry
matched_geometry = match_data["matchings"][0]["geometry"]["coordinates"]

# Convert [lon,lat] → [lat,lon]
matched_latlon = [[lat, lon] for lon, lat in matched_geometry]

# 4️⃣ Draw smooth BLUE route
folium.PolyLine(
    locations=matched_latlon,
    color="blue",
    weight=6,
    opacity=0.9,
    tooltip="OSRM Matched Route"
).add_to(folium_map)

folium_map


## Conclusion

In this notebook, we explored the OSRM **Nearest** service
and its role in aligning geographic coordinates with
the routable street network.

We demonstrated:

- Snapping a single coordinate to the nearest road segment  
- Retrieving multiple proximity-based waypoint candidates  
- Converting returned geometries into structured `Point` objects  
- Visualizing snapped locations on an interactive map  
- Performing batch snapping using a CSV dataset  
- Reconstructing a continuous path using the OSRM Match service  

The Nearest service plays a critical preprocessing role
in routing workflows. Since routing algorithms operate
on graph-based road representations, input coordinates
must first be projected onto valid network nodes.

By integrating snapping, batch processing, and map matching,
we established a complete pipeline for transforming raw
GPS coordinates into graph-aligned routing inputs.